In [1]:
import pandas as pd
from siuba import *
import numpy as np

import altair as alt

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

from calitp import to_snakecase

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You m

In [2]:
df=pd.read_csv('gs://calitp-analytics-data/data-analyses/5310/5310_org_info_manualfill.csv')

In [3]:
df.sample(5)

,organization_name,ntd_id,itp_id,name_NTD_Airtable,doing_business_as,reporter_acronym,organization_type,reporter_type,city,county,mobility_services_operated,merge_status
132,Lake Links Inc,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,NaN,Clearlake,Lake,NaN,No Match
79,Humboldt Medi-Trans,NaN,NaN,Humboldt Medi-Trans Inc,NaN,NaN,Non-Profit Organization,NaN,McKinleyville,Humboldt,Humboldt Medi-Trans,"Manual Match, Airtable"
10,City of Santa Rosa,90017,301.0,City of Santa Rosa,NaN,NaN,"City, County or Local Government Unit or Depar...",Full Reporter,Santa Rosa,Sonoma,"Santa Rosa CityBus,Santa Rosa Call Ahead Service",Name Match; NTD
64,Sunline Transit Agency,90079,327.0,SunLine Transit Agency,NaN,NaN,Independent Public Agency or Authority of Tran...,Full Reporter,Thousand Palms,Riverside,"SunLine Transit,SunLine School Tripper,SunDial...","Manual Match, NTD"
119,Calaveras County Public Works,NaN,NaN,NaN,NaN,NaN,"City, County or Local Government Unit or Depar...",NaN,San Andreas,Calaveras,NaN,No Match


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   organization_name           147 non-null    object 
 1   ntd_id                      37 non-null     object 
 2   itp_id                      78 non-null     float64
 3   name_NTD_Airtable           108 non-null    object 
 4   doing_business_as           7 non-null      object 
 5   reporter_acronym            28 non-null     object 
 6   organization_type           147 non-null    object 
 7   reporter_type               37 non-null     object 
 8   city                        146 non-null    object 
 9   county                      146 non-null    object 
 10  mobility_services_operated  90 non-null     object 
 11  merge_status                147 non-null    object 
dtypes: float64(1), object(11)
memory usage: 13.9+ KB


In [5]:
df>>count(_.county)>>arrange(-_.n)

,county,n
32,Riverside,10
0,Alameda,9
35,San Bernardino,9
9,Contra Costa,8
38,San Francisco,7
19,Kern,6
24,Mendocino,5
51,Solano,5
33,Sacramento,4
37,San Diego,4


In [6]:
df>>filter(_.county.str.contains(", "))

,organization_name,ntd_id,itp_id,name_NTD_Airtable,doing_business_as,reporter_acronym,organization_type,reporter_type,city,county,mobility_services_operated,merge_status
2,Area 1 Agency on Aging,NaN,399.0,Area 1 Agency on Aging,NaN,NaN,Non-Profit Organization,NaN,Eureka,"Humboldt, De Norte",NaN,Name Match; Airtable
3,Big Valley 50 Plus,NaN,404.0,Big Valley 50 Plus,NaN,NaN,Non-Profit Organization,NaN,Beiber,"Lassen, Modoc",Big Valley 50 Plus,Name Match; Airtable
13,Common Ground Senior Services,NaN,419.0,Common Ground Senior Services,NaN,NaN,Non-Profit Organization,NaN,San Andreas,"Calaveras, Amador, Tuolumne",Silver Streak Transport,Name Match; Airtable
19,Friends of Children with Special Needs,NaN,NaN,Friends of Children with Special Needs,NaN,NaN,Non-Profit Organization,NaN,Fremont,"Santa Clara, Alameda",NaN,Name Match; Airtable
24,Inyo-Mono Association for the Handicapped,NaN,434.0,Inyo-Mono Association for the Handicapped,NaN,NaN,Non-Profit Organization,NaN,Bishop,"Inyo, Mono",Inyo-Mono Association for the Handicapped,Name Match; Airtable
51,Self-Help for the Elderly,NaN,NaN,Self-Help for the Elderly,NaN,NaN,Non-Profit Organization,NaN,San Francisco,"San Francisco, Alameda, San Mateo, Santa Clara",Self-Help for the Elderly,Name Match; Airtable
52,Tahoe Transportation District,91092,331.0,Tahoe Transportation District,NaN,TTD,Independent Public Agency or Authority of Tran...,Full Reporter,Zephyr Cove,"Douglas, Placer, Washoe, Carson City","Tahoe Truckee Area Regional Transportation,Tah...",Name Match; Airtable
53,The Arc of Amador and Calaveras,NaN,461.0,The Arc of Amador and Calaveras,NaN,NaN,Non-Profit Organization,NaN,Sutter Creek,"Amador, Calaveras",The Arc of Amador and Calaveras,Name Match; Airtable
74,Community Bridges Liftline,NaN,420.0,Community Bridges Inc,NaN,NaN,Non-Profit Organization,NaN,Watsonville,"Monterey, San Mateo, San Benito, Santa Clara, ...",Lift Line,"Manual Match, Airtable"
80,Lutheran Social Services of Southern California,NaN,439.0,Lutheran Social Services,NaN,NaN,Non-Profit Organization,NaN,Orange,"Orange, Los Angeles, Riverside, San Bernardino...",NaN,"Manual Match, Airtable"


In [7]:
df>>count(_.organization_type)>>arrange(-_.n)

,organization_type,n
5,Non-Profit Organization,90
3,Independent Public Agency or Authority of Tran...,28
0,"City, County or Local Government Unit or Depar...",12
2,Independent Agency,5
1,Company,4
4,MPO/RTPA,3
6,Other,3
7,Private-Non-Profit Corporation,1
8,University,1


## Charts

In [8]:
df.sample()

,organization_name,ntd_id,itp_id,name_NTD_Airtable,doing_business_as,reporter_acronym,organization_type,reporter_type,city,county,mobility_services_operated,merge_status
135,Tuolumne County Transit Agency (TCTA),9R02-91057,NaN,Tuolumne County Transit,NaN,TCT,Independent Public Agency or Authority of Tran...,Rural Reporter,Sonora,Tuolumne,NaN,"Manual Match, NTD"


In [9]:
def labeling(word):
    LABEL_DICT = {
        "dist": "District",
        "nunique": "Number of Unique",
        "n":"Count",
        "name_NTD_Airtable":"Organization Name"
    }

    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        #word = word.replace("n_", "Number of ").title()
        word = word.replace("unique_", "Number of Unique ").title()
        word = word.replace("_", " ").title()

    return word

In [10]:
def bar_chart(df, x_col, y_col, color_col, chart_title=""):

    bar = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X(x_col, title=labeling(x_col), sort=("-y")),
            y=alt.Y(y_col, title=labeling(y_col)),
            color=alt.Color(
                color_col,
                scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
                legend=alt.Legend(title=(labeling(color_col)), symbolLimit=10)
            )
        ).properties(title=chart_title))
    
    chart = styleguide.preset_chart_config(bar)
    
    return chart

In [11]:
org_counts = df>>count(_.organization_type)

In [12]:
org_counts>>arrange(-_.n)

,organization_type,n
5,Non-Profit Organization,90
3,Independent Public Agency or Authority of Tran...,28
0,"City, County or Local Government Unit or Depar...",12
2,Independent Agency,5
1,Company,4
4,MPO/RTPA,3
6,Other,3
7,Private-Non-Profit Corporation,1
8,University,1


In [13]:
bar_chart(org_counts,
    "organization_type",
    "n",
    "organization_type",
    "5310 Organization Types"
)

alt.Chart(...)

In [14]:
def charttext(df, x_col, y_col, color_col):
    bars = alt.Chart(df).mark_bar().encode(
            x=alt.X(x_col, title=labeling(x_col), sort=("-y")),
            y=alt.Y(y_col, title=labeling(y_col)),
            color=alt.Color(
                color_col,
                scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
                legend=alt.Legend(title=(labeling(color_col)), symbolLimit=10)
    ))

    text = bars.mark_text(
        align="left", baseline="middle", color="black", dy=3
    ).encode(
        text= y_col, color=alt.value("black")
    )
    
    display(styleguide.preset_chart_config(bars + text))

In [15]:
charttext(org_counts, "organization_type", "n", "organization_type")

alt.LayerChart(...)

In [16]:
## create long df for county counts

In [17]:
df.sample(5)

,organization_name,ntd_id,itp_id,name_NTD_Airtable,doing_business_as,reporter_acronym,organization_type,reporter_type,city,county,mobility_services_operated,merge_status
44,Redwood Coast Transit Authority,9R02-91097,261.0,Redwood Coast Transit Authority,NaN,RCTA,Independent Public Agency or Authority of Tran...,Rural Reporter,Crescent City,Del Norte,"Redwood Coast Transit,Redwood Coast Dial-A-Ride",Name Match; NTD
10,City of Santa Rosa,90017,301.0,City of Santa Rosa,NaN,NaN,"City, County or Local Government Unit or Depar...",Full Reporter,Santa Rosa,Sonoma,"Santa Rosa CityBus,Santa Rosa Call Ahead Service",Name Match; NTD
100,United Cerebral Palsy Association of Greater S...,NaN,464.0,United Cerebral Palsy Association of Greater S...,NaN,NaN,Non-Profit Organization,NaN,Sacramento,Sacramento,UCP Safe Trip,"Manual Match, Airtable"
109,County of Sonoma Human Services Department Adu...,NaN,NaN,NaN,NaN,NaN,Independent Agency,NaN,Sonoma,Sonoma,NaN,No Match
78,Faith in Action,NaN,NaN,Faith in Action Inc,NaN,NaN,Non-Profit Organization,NaN,Vacaville,Solano,Ride with Pride,"Manual Match, Airtable"


In [18]:
single_county = df[~df["county"].str.contains(",", na=False)]

In [19]:
single_county>>count(_.county)>>arrange(-_.n)

,county,n
20,Riverside,10
0,Alameda,9
23,San Bernardino,9
5,Contra Costa,8
25,San Francisco,7
11,Kern,6
15,Mendocino,5
33,Solano,5
21,Sacramento,4
24,San Diego,4


In [20]:
bar_chart((single_county>>count(_.county)),
    "county",
    "n",
    "county",
    "Counties with 5310 Organizations (Single County)"
)

alt.Chart(...)

In [21]:
df1 = df>>select(_.organization_name, _.ntd_id, _.itp_id, _.organization_type, _.city, _.county, _.merge_status)

In [22]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   organization_name  147 non-null    object 
 1   ntd_id             37 non-null     object 
 2   itp_id             78 non-null     float64
 3   organization_type  147 non-null    object 
 4   city               146 non-null    object 
 5   county             146 non-null    object 
 6   merge_status       147 non-null    object 
dtypes: float64(1), object(6)
memory usage: 8.2+ KB


In [23]:
df1.explode('county')

,organization_name,ntd_id,itp_id,organization_type,city,county,merge_status
0,Angel View Inc,NaN,NaN,Non-Profit Organization,Cathedral City,Riverside,Name Match; Airtable
1,ARC Imperial Valley,NaN,398.0,Non-Profit Organization,El Centro,Imperial,Name Match; Airtable
2,Area 1 Agency on Aging,NaN,399.0,Non-Profit Organization,Eureka,"Humboldt, De Norte",Name Match; Airtable
3,Big Valley 50 Plus,NaN,404.0,Non-Profit Organization,Beiber,"Lassen, Modoc",Name Match; Airtable
4,Care-A-Van Transit Systems Inc,NaN,407.0,Company,Hemet,Riverside,Name Match; Airtable
...,...,...,...,...,...,...,...
142,Loma Linda University Health,NaN,NaN,Other,Loma Linda,San Bernardino,No Match
143,New Advances for People with Disabilities,NaN,NaN,Non-Profit Organization,Bakersfield,Kern,No Match
144,Pomeroy Recreation & Rehabilitation Center,NaN,NaN,Non-Profit Organization,San Francisco,San Francisco,No Match
145,Regents of the University of California Santa ...,NaN,NaN,University,Santa Cruz,Santa Cruz,No Match


In [24]:
## from https://medium.com/swlh/three-routes-convert-comma-separated-column-to-rows-c17c85079ecf

df2 = df1.assign(county_split = df['county'].str.split(', ')).drop("county", axis=1)\
    .rename(columns = {"county_split": "county"})


In [25]:
df2

,organization_name,ntd_id,itp_id,organization_type,city,merge_status,county
0,Angel View Inc,NaN,NaN,Non-Profit Organization,Cathedral City,Name Match; Airtable,[Riverside]
1,ARC Imperial Valley,NaN,398.0,Non-Profit Organization,El Centro,Name Match; Airtable,[Imperial]
2,Area 1 Agency on Aging,NaN,399.0,Non-Profit Organization,Eureka,Name Match; Airtable,"[Humboldt, De Norte]"
3,Big Valley 50 Plus,NaN,404.0,Non-Profit Organization,Beiber,Name Match; Airtable,"[Lassen, Modoc]"
4,Care-A-Van Transit Systems Inc,NaN,407.0,Company,Hemet,Name Match; Airtable,[Riverside]
...,...,...,...,...,...,...,...
142,Loma Linda University Health,NaN,NaN,Other,Loma Linda,No Match,[San Bernardino]
143,New Advances for People with Disabilities,NaN,NaN,Non-Profit Organization,Bakersfield,No Match,[Kern]
144,Pomeroy Recreation & Rehabilitation Center,NaN,NaN,Non-Profit Organization,San Francisco,No Match,[San Francisco]
145,Regents of the University of California Santa ...,NaN,NaN,University,Santa Cruz,No Match,[Santa Cruz]


In [26]:
df2 = df2.explode('county')

In [27]:
df2>>count(_.county)>>arrange(-_.n)

,county,n
0,Alameda,12
29,Riverside,12
32,San Bernardino,11
6,Contra Costa,10
34,San Francisco,10
39,Santa Clara,9
15,Kern,6
37,San Mateo,6
43,Solano,6
21,Mendocino,5


In [28]:
bar_chart((df2>>count(_.county)),
    "county",
    "n",
    "county",
    "Counties with 5310 Organizations Operating"
)

alt.Chart(...)

In [29]:
(df2>>count(_.county)).mean()

/tmp/ipykernel_1084/1613865003.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


n    3.5
dtype: float64

In [30]:
(df2>>group_by(_.organization_type)>>count(_.county)>>arrange(-_.n)).head(20)

,organization_type,county,n
51,Non-Profit Organization,Alameda,11
55,Non-Profit Organization,Contra Costa,8
79,Non-Profit Organization,San Francisco,8
84,Non-Profit Organization,Santa Clara,8
74,Non-Profit Organization,Riverside,7
77,Non-Profit Organization,San Bernardino,6
61,Non-Profit Organization,Kern,5
82,Non-Profit Organization,San Mateo,5
67,Non-Profit Organization,Mendocino,4
75,Non-Profit Organization,Sacramento,4


### Counts

In [31]:
len(df1>>filter(_.ntd_id.notnull()))

37

In [32]:
(df1>>filter(_.ntd_id.notnull())).organization_type.value_counts()

Independent Public Agency or Authority of Transit Service                25
City, County or Local Government Unit or Department of Transportation     8
MPO/RTPA                                                                  3
Private-Non-Profit Corporation                                            1
Name: organization_type, dtype: int64

In [33]:
(df1>>filter(_.ntd_id.notnull())>>filter(_.organization_type=="Private-Non-Profit Corporation"))

,organization_name,ntd_id,itp_id,organization_type,city,county,merge_status
66,Easy Lift Transportation Inc,90243,100.0,Private-Non-Profit Corporation,Goleta,Santa Barbara,"Manual Match, NTD"


In [34]:
df1.organization_type.value_counts()

Non-Profit Organization                                                  90
Independent Public Agency or Authority of Transit Service                28
City, County or Local Government Unit or Department of Transportation    12
Independent Agency                                                        5
Company                                                                   4
MPO/RTPA                                                                  3
Other                                                                     3
Private-Non-Profit Corporation                                            1
University                                                                1
Name: organization_type, dtype: int64

In [35]:
print(len(df1.query("organization_type==('Independent Public Agency or Authority of Transit Service') or organization_type==('City, County or Local Government Unit or Department of Transportation')")))


df1.query("organization_type==('Independent Public Agency or Authority of Transit Service') or organization_type==('City, County or Local Government Unit or Department of Transportation')")


40


,organization_name,ntd_id,itp_id,organization_type,city,county,merge_status
7,City of Fontana,NaN,414.0,"City, County or Local Government Unit or Depar...",Fontana,San Bernardino,Name Match; Airtable
8,City of Rio Vista,9R02-91014,264.0,"City, County or Local Government Unit or Depar...",Rio Vista,Solano,Name Match; NTD
9,City of Roseville,90168,271.0,"City, County or Local Government Unit or Depar...",Roseville,Placer,Name Match; NTD
10,City of Santa Rosa,90017,301.0,"City, County or Local Government Unit or Depar...",Santa Rosa,Sonoma,Name Match; NTD
11,City of Visalia,90091,361.0,"City, County or Local Government Unit or Depar...",Visalia,Tulare,Name Match; NTD
12,Colusa County Transit Agency,9R02-91112,74.0,"City, County or Local Government Unit or Depar...",Colusa,Colusa,Name Match; NTD
16,Eastern Sierra Transit Authority,9R02-91062,99.0,Independent Public Agency or Authority of Tran...,Bishop,Inyo,Name Match; NTD
17,El Dorado County Transit Authority,90229,101.0,Independent Public Agency or Authority of Tran...,Diamond Springs,El Dorado,Name Match; NTD
21,Humboldt Transit Authority,9R02-91036,135.0,Independent Public Agency or Authority of Tran...,Eureka,Humboldt,Name Match; NTD
22,Imperial County Transportation Commission,90226,138.0,Independent Public Agency or Authority of Tran...,El Centro,Imperial,Name Match; NTD


In [36]:
print(len(df1.query("organization_name.str.contains('Transit Authority') or organization_name.str.contains('Transit Agency') or organization_name.str.contains('Transportation Authority') or organization_name.str.contains('Transportation Agency') or organization_name.str.contains('Transportation District') or organization_name.str.contains('Transit District')")
))

df1.query("organization_name.str.contains('Transit Authority') or organization_name.str.contains('Transit Agency') or organization_name.str.contains('Transportation Authority') or organization_name.str.contains('Transportation Agency') or organization_name.str.contains('Transportation District') or organization_name.str.contains('Transit District')")



25


,organization_name,ntd_id,itp_id,organization_type,city,county,merge_status
12,Colusa County Transit Agency,9R02-91112,74.0,"City, County or Local Government Unit or Depar...",Colusa,Colusa,Name Match; NTD
16,Eastern Sierra Transit Authority,9R02-91062,99.0,Independent Public Agency or Authority of Tran...,Bishop,Inyo,Name Match; NTD
17,El Dorado County Transit Authority,90229,101.0,Independent Public Agency or Authority of Tran...,Diamond Springs,El Dorado,Name Match; NTD
21,Humboldt Transit Authority,9R02-91036,135.0,Independent Public Agency or Authority of Tran...,Eureka,Humboldt,Name Match; NTD
25,Lake Transit Authority,9R02-91053,159.0,Independent Public Agency or Authority of Tran...,Lower Lake,Lake,Name Match; NTD
27,Marin County Transit District,90234,194.0,Independent Public Agency or Authority of Tran...,San Rafael,Marin,Name Match; NTD
28,Mendocino Transit Authority,9R02-91047,198.0,Independent Public Agency or Authority of Tran...,Ukiah,Mendocino,Name Match; NTD
32,Morongo Basin Transit Authority,9R02-91090,212.0,Independent Public Agency or Authority of Tran...,Joshua Tree,San Bernardino,Name Match; NTD
34,Napa Valley Transportation Authority,90088,218.0,Independent Public Agency or Authority of Tran...,Napa,Napa,Name Match; NTD
39,Palo Verde Valley Transit Agency,9R02-99454,238.0,Independent Public Agency or Authority of Tran...,Blythe,Riverside,Name Match; NTD


## Adding more information for 5310 agencies from BlackCat

In [37]:
bc = to_snakecase(
    pd.read_excel(
        "gs://calitp-analytics-data/data-analyses/grants/Grant+Projects_3_17_2022.xlsx"
    )
)
bc = bc>>filter(_.funding_program.str.contains('5310'))
bc["organization_name"] = (
    bc["organization_name"]
    .str.replace(",", "")
    .str.replace(".", "")
    .str.replace("/", " ")
    #.str.replace("\\", "")
)

/tmp/ipykernel_1084/4035653396.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [38]:
bc.head(5)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,project_closed_by,project_closed_date,project_closed_time
125,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Amador Transit,BCG0001035,Operating Assistance,300901,64AO18-00677,30144.0,30144.0,30144.0,0.0,0,Open,NaN,NaN,NaN
126,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Angel View Inc,BCG0000324,Operating Assistance,300901,64AO18-00770,125000.0,125000.0,125000.0,0.0,0,Open,NaN,NaN,NaN
127,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,ARC Imperial Valley,BCG0000988,Operating Assistance,300901,64AO18-00630,193211.0,193211.0,193211.0,0.0,0,Open,NaN,NaN,NaN
128,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,ARC Imperial Valley,BCG0002757,Operating Assistance,300901,64AO18-00630,40000.0,40000.0,40000.0,0.0,0,Open,NaN,NaN,NaN
129,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Area 1 Agency on Aging,BCG0000392,Operating Assistance,300901,64AO18-00631,146534.0,146534.0,146534.0,0.0,0,Open,NaN,NaN,NaN


In [39]:
bc>>filter(_.organization_name=="Amador Transit")

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,project_closed_by,project_closed_date,project_closed_time
125,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Amador Transit,BCG0001035,Operating Assistance,300901,64AO18-00677,30144.0,30144.0,30144.0,0.0,0,Open,NaN,NaN,NaN
206,2017,5310 Trad,CA-2017-169 | 0017000186-T,2017,Amador Transit,BCG0000986,Mobility Management,117L00,64AM18-00928,170998.0,170998.0,170998.0,0.0,0,Open,NaN,NaN,NaN
1120,2019,5310 Trad,CA-2020-244 | 0020000273-T,2019,Amador Transit,BCG0002181,Mobility Management,117L00,64AM19-01130,53771.0,268853.0,268853.0,-215082.0,0,Open,NaN,NaN,NaN
1121,2019,5310 Trad,CA-2020-244 | 0020000273-T,2019,Amador Transit,BCG0002181,Mobility Management,117L00,64AM19-01130,215082.0,268853.0,268853.0,-53771.0,0,Open,NaN,NaN,NaN


In [40]:
bc>>filter(_.funding_program=="5310 Exp")

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,project_closed_by,project_closed_date,project_closed_time
125,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Amador Transit,BCG0001035,Operating Assistance,300901,64AO18-00677,30144.0,30144.0,30144.0,0.0,0,Open,NaN,NaN,NaN
126,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Angel View Inc,BCG0000324,Operating Assistance,300901,64AO18-00770,125000.0,125000.0,125000.0,0.0,0,Open,NaN,NaN,NaN
127,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,ARC Imperial Valley,BCG0000988,Operating Assistance,300901,64AO18-00630,193211.0,193211.0,193211.0,0.0,0,Open,NaN,NaN,NaN
128,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,ARC Imperial Valley,BCG0002757,Operating Assistance,300901,64AO18-00630,40000.0,40000.0,40000.0,0.0,0,Open,NaN,NaN,NaN
129,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Area 1 Agency on Aging,BCG0000392,Operating Assistance,300901,64AO18-00631,146534.0,146534.0,146534.0,0.0,0,Open,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,2019,5310 Exp,CA-2020-244 | 0020000273-E,2019,United Cerebral Palsy Association of Greater S...,BCG0002138,Operating Assistance,300901,64AO19-01244,233813.0,467625.0,0.0,233813.0,0,Open,NaN,NaN,NaN
1106,2019,5310 Exp,CA-2020-244 | 0020000273-E,2019,Valley Resource Center,BCG0002164,Operating Assistance,300901,64AO19-01249,67877.0,135754.0,0.0,67877.0,0,Open,NaN,NaN,NaN
1107,2019,5310 Exp,CA-2020-244 | 0020000273-E,2019,West Valley Community Services,BCG0002607,Operating Assistance,300901,NaN,162704.0,0.0,0.0,162704.0,0,Open,NaN,NaN,NaN
1108,2019,5310 Exp,CA-2020-244 | 0020000273-E,2019,West Valley Community Services,BCG0002607,Operating Assistance,300901,NaN,162705.0,0.0,0.0,162705.0,0,Open,NaN,NaN,NaN


In [41]:
(
    bc
    >> group_by(_.ali)
    >> summarize(n=_.organization_name.nunique())
    >> arrange(-_.n)
    >> filter(_.n > 1)
)

,ali,n
14,300901,66
0,111204,63
13,117L00,35
1,111215,24
4,111304,23
5,111315,19
6,114207,9
11,116203,7
9,114220,5
7,114208,4


In [42]:
p = pd.merge(bc, df1, how='left', on='organization_name')

In [43]:
p.sample(5)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,...,project_status,project_closed_by,project_closed_date,project_closed_time,ntd_id,itp_id,organization_type,city,county,merge_status
1030,2019,5310 Trad,CA-2020-244 | 0020000273-T,2019,United Cerebral Palsy Association of Greater S...,BCG0002049,Purchase Replacement < 30 Ft Bus,111204,64AC19-01243,16240.0,...,Open,NaN,NaN,NaN,NaN,464.0,Non-Profit Organization,Sacramento,Sacramento,"Manual Match, Airtable"
326,2017,5310 Trad,CA-2017-169 | 0017000186-T,2017,Monterey-Salinas Transit,BCG0000707,Buy <30-Ft Bus For Expansion,111304,64AC18-00645,70000.0,...,Open,NaN,NaN,NaN,90062,208.0,Independent Public Agency or Authority of Tran...,Monterey,Monterey,Name Match; NTD
874,2019,5310 Trad,CA-2020-244 | 0020000273-T,2019,On Lok Senior Health Services,BCG0001908,Purchase Replacement < 30 Ft Bus,111204,64AC19-01207,14240.0,...,Open,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,San Francisco,San Francisco,"Manual Match, Airtable"
240,2017,5310 Trad,CA-2017-169 | 0017000186-T,2017,Help Central Inc,BCG0000627,211 One Stop Call Center & Travel Training - M...,117L00,64AM18-00670,457453.0,...,Open,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,Chico,"Butte, Glen",No Match
1060,2019,5310 Trad,CA-2020-244 | 0020000273-T,2019,United Cerebral Palsy of San Luis Obispo County,BCG0001735,Purchase Replacement < 30 Ft Bus,111204,64AC19-01245,-115.0,...,Open,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,San Luis Obispo,San Luis Obispo,"Manual Match, Airtable"


In [58]:
p['description'] = p['description'].str.strip()

In [59]:
(
    p
    >> group_by(_.organization_name, _.funding_program)
    >> summarize(n=_.description.nunique())
    >> arrange(-_.n)
    >> filter(_.n > 1)
)

,organization_name,funding_program,n
39,Community Bridges Liftline,5310 Trad,6
181,United Cerebral Palsy Association of Greater S...,5310 Trad,6
139,Porterville Sheltered Workshop,5310 Trad,5
11,Asian Community Center of Sacramento Valley In...,5310 Trad,4
21,Catholic Charities of the Diocese of Stockton,5310 Trad,4
...,...,...,...
173,The Center for Independent Living,5310 Trad,2
185,Valley Resource Center,5310 Trad,2
187,Victor Valley Community Services Council,5310 Trad,2
188,Victor Valley Transit Authority,5310 Trad,2


In [60]:
p>>count(_.grant_fiscal_year)

,grant_fiscal_year,n
0,2017,513
1,2019,602


In [61]:
p>>count(_.project_year)

,project_year,n
0,2017,507
1,2018,4
2,2019,604


In [62]:
p>>filter(_.project_year==2018)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,...,project_status,project_closed_by,project_closed_date,project_closed_time,ntd_id,itp_id,organization_type,city,county,merge_status
72,2017,5310 Exp,CA-2017-169 | 0017000186-E,2018,County of Sonoma Human Services Department Adu...,BCG0000526,Operating Assistance,300901,64AO18-00794,266975.0,...,Open,NaN,NaN,NaN,NaN,NaN,Independent Agency,Sonoma,Sonoma,No Match
73,2017,5310 Exp,CA-2017-169 | 0017000186-E,2018,Desert Access and Mobility Inc,BCG0000752,Operating Assistance,300901,64AO16-00777,125000.0,...,Open,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,Palm Springs,Riverside,Name Match; Airtable
510,2017,5310 Trad,CA-2017-169 | 0017000186-T,2018,County of Sonoma Human Services Department Adu...,BCG0000525,Mobility Management,117L00,64AM18-00764,157532.0,...,Open,NaN,NaN,NaN,NaN,NaN,Independent Agency,Sonoma,Sonoma,No Match
511,2017,5310 Trad,CA-2017-169 | 0017000186-T,2018,County of Sonoma Human Services Department Adu...,BCG0000598,Purchase Computer Software,114208,64AM18-00764,10000.0,...,Open,NaN,NaN,NaN,NaN,NaN,Independent Agency,Sonoma,Sonoma,No Match


In [63]:
p>>count(_.grant_number)


,grant_number,n
0,CA-2017-169 | 0017000186-E,75
1,CA-2017-169 | 0017000186-T,438
2,CA-2020-244 | 0020000273-E,76
3,CA-2020-244 | 0020000273-T,526


In [72]:
(p>>group_by(_.organization_name, _.grant_fiscal_year)>>count(_.description))

,organization_name,grant_fiscal_year,description,n
0,ARC Bakersfield,2017,Purchase Replacement < 30 Ft Bus,16
1,ARC Bakersfield,2017,Purchase Replacement Van,4
2,ARC Imperial Valley,2017,Operating Assistance,2
3,ARC Imperial Valley,2019,Buy Van For Svc Expansion,4
4,ARC Imperial Valley,2019,Operating Assistance,2
...,...,...,...,...
353,Wilshire Community Services,2019,Operating Assistance,1
354,Work Training Center,2017,Purchase Radios,1
355,Work Training Center,2017,Purchase Replacement < 30 Ft Bus,6
356,Yolo County Transportation District,2017,Mobility Management,2


In [65]:
p>>group_by(_.organization_name)>>count(_.description)

,organization_name,description,n
0,ARC Bakersfield,Purchase Replacement < 30 Ft Bus,16
1,ARC Bakersfield,Purchase Replacement Van,4
2,ARC Imperial Valley,Buy Van For Svc Expansion,4
3,ARC Imperial Valley,Operating Assistance,4
4,Able Industries,Buy <30-Ft Bus For Expansion,6
...,...,...,...
280,Wilshire Community Services,Operating Assistance,2
281,Work Training Center,Purchase Radios,1
282,Work Training Center,Purchase Replacement < 30 Ft Bus,6
283,Yolo County Transportation District,Mobility Management,2


In [66]:
p_agg = (p.groupby(['organization_name'])
      .agg({'description': lambda x: x.tolist()})
      .rename({'description' : 'descriptions'},axis=1)
      .reset_index())

In [74]:
p_agg

,organization_name,descriptions
0,ARC Bakersfield,"[Purchase Replacement < 30 Ft Bus, Purchase Re..."
1,ARC Imperial Valley,"[Operating Assistance, Operating Assistance, O..."
2,Able Industries,"[Buy <30-Ft Bus For Expansion, Buy <30-Ft Bus ..."
3,Alegria Community Living,"[Purchase Replacement Van, Purchase Replacemen..."
4,Amador Transit,"[Operating Assistance, Mobility Management, Mo..."
...,...,...
142,Vivalon Inc,"[Operating Assistance, Purchase Replacement < ..."
143,West Valley Community Services,"[Operating Assistance, Operating Assistance, O..."
144,Wilshire Community Services,"[Operating Assistance, Operating Assistance]"
145,Work Training Center,"[Purchase Replacement < 30 Ft Bus, Purchase Re..."


In [68]:
p.description.value_counts()

Purchase Replacement < 30 Ft Bus                                                                464
Buy <30-Ft Bus For Expansion                                                                    138
Purchase Replacement Van                                                                        122
Operating Assistance                                                                            113
Buy Van For Svc Expansion                                                                        93
Mobility Management                                                                              72
Purchase Computer Hardware                                                                       16
Purchase Radios                                                                                  16
Surveillance/Security                                                                            10
Support Equip/Facilities-Equipment                                                                8


In [69]:
p.sample()

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,...,project_status,project_closed_by,project_closed_date,project_closed_time,ntd_id,itp_id,organization_type,city,county,merge_status
500,2017,5310 Trad,CA-2017-169 | 0017000186-T,2017,Vivalon Inc,BCG0000530,Purchase Replacement < 30 Ft Bus,111204,64AC18-00706,70000.0,...,Open,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,San Rafael,Marin,Name Match; Airtable


In [70]:
(
    p
    >> group_by(_.organization_name, _.organization_type)
    >> summarize(nunique_desc=_.description.nunique())
    >> arrange(-_.nunique_desc)
)

,organization_name,organization_type,nunique_desc
28,Community Bridges Liftline,Non-Profit Organization,7
35,County of Sonoma Human Services Department Adu...,Independent Agency,7
136,United Cerebral Palsy Association of Greater S...,Non-Profit Organization,7
85,Mobility Matters,Non-Profit Organization,6
7,Asian Community Center of Sacramento Valley In...,Non-Profit Organization,5
...,...,...,...
135,United Cerebral Palsy Association Inc of Stani...,Non-Profit Organization,1
137,United Cerebral Palsy of San Luis Obispo County,Non-Profit Organization,1
138,Valley Achievement Center,Non-Profit Organization,1
143,West Valley Community Services,Non-Profit Organization,1
